# Finetuen T5

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. Load data
df = pd.read_csv('dataset/subtask1_train_batch2_preprocess.csv')
df_subset = df.iloc[100:].reset_index(drop=True)

# 2. Label encoding
label2id = {'contra': 0, 'unver': 1, 'entail': 2}
df_subset = df_subset[df_subset['label'].isin(label2id)]  # Only keep three classes
df_subset['label_id'] = df_subset['label'].map(label2id)

# 3. Split train/validation sets
train_df, eval_df = train_test_split(df_subset, test_size=0.1, random_state=42, stratify=df_subset['label_id'])

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base', trust_remote_code=True)  #
# 4. Build Dataset using datasets library
from datasets import Dataset
def tokenize_function(examples):
    return tokenizer(
        examples['reference'],
        examples['claim'],
        truncation=True,
        padding='max_length',
        max_length=512
    )
# Convert pandas dataframes to datasets
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)
# Apply tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)
# Rename label column and set format
train_dataset = train_dataset.rename_column('label_id', 'labels')
eval_dataset = eval_dataset.rename_column('label_id', 'labels')
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
eval_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/1792 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [6]:
model = AutoModelForSequenceClassification.from_pretrained(
    'roberta-base', 
    num_labels=3  # contra, unver, entail
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
import numpy as np
import evaluate
from transformers import Trainer, TrainingArguments

# 1. Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True
)

# 2. Define a compute metrics function
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# 3. Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
# 4. Start fine-tuning
trainer.train()

# 5. Evaluate the model
results = trainer.evaluate()
print(results)

/tmp/ipykernel_20077/1910821690.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
print(train_dataset[0])